In [ ]:
# Run setup notebook from here
!git clone https://github.com/goodarzi64/GHI_Forecasting
%cd GHI_Forecasting
!git pull
%run /content/GHI_Forecasting/notebooks/00_colab_setup.ipynb
from google.colab import drive
drive.mount("/content/gdrive")
%run /content/GHI_Forecasting/notebooks/01_import_datasets.ipynb

In [ ]:
from src.temporal_autoencoder import pretrain_en_de_with_regularizers

# make sure tensors are float32 before training
temporal_node_tensor = torch.as_tensor(temporal_node_tensor, dtype=torch.float32, device=device)

mask_embed = torch.as_tensor(masks["mask_embed"], dtype=torch.bool, device=device)
mask_cloud = torch.as_tensor(masks["mask_cloud"], dtype=torch.bool, device=device)

model, logs, best_epoch = pretrain_en_de_with_regularizers(
    train_tensor=temporal_node_tensor,
    val_tensor=None,
    in_dim=mask_embed.sum().item(),
    embed_dim=16,
    conv_hidden=128,
    window=6,
    use_attention=True,
    batch_size=8,
    lr=1e-3,
    epochs=20,
    device=device,
    early_stopping_patience=5,
    mask_embed=mask_embed,
    mask_cloud=mask_cloud,
    save_path=None,
    verbose=True,
)


In [ ]:
from src.cv_splits import make_expanding_folds, extract_fold_data

T = temporal_node_tensor.shape[0]
first_train_end = int(T * 0.33)
val_window = int(T * 0.11)
n_folds = 3

folds = make_expanding_folds(
    T=T,
    train_start=0,
    first_train_end=first_train_end,
    val_window=val_window,
    n_folds=n_folds,
)

print("FOLD SPLITS:")
for i, f in enumerate(folds):
    print(f"Fold {i+1}: Train {f['train_slice']}  Val {f['val_slice']}")

train_data, val_data = extract_fold_data(temporal_node_tensor, folds, fold_idx=0)


FOLD SPLITS:
Fold 1: Train (0, 7588)  Val (7588, 10117)
Fold 2: Train (0, 10117)  Val (10117, 12646)
Fold 3: Train (0, 12646)  Val (12646, 15175)


In [ ]:
from src.hparam_search_encoder import HParamConfig, run_hparam_search, select_best

cfg = HParamConfig(
    embed_dims=[8, 16, 32],
    conv_hiddens=[32, 64, 128, 256],
    seeds=[123],
    folds=[0, 1, 2],
    window=12,
    use_attention=True,
    batch_size=8,
    epochs=1,
    lr=1e-3,
    early_stopping_patience=5,
)

mask_embed = torch.as_tensor(masks["mask_embed"], dtype=torch.bool, device=device)
mask_cloud = torch.as_tensor(masks["mask_cloud"], dtype=torch.bool, device=device)

results = run_hparam_search(
    temporal_node_tensor=temporal_node_tensor,
    folds=folds,
    mask_embed=mask_embed,
    mask_cloud=mask_cloud,
    base_dir="/content/gdrive/MyDrive/hparam_search_encoder",
    cfg=cfg,
    device=device,
    verbose=True,
)


best_cfg = select_best(results)
print(best_cfg)


In [ ]:
from src.hparam_results import collect_hparam_records, summarize_hparam_results

base_dir = "/content/gdrive/MyDrive/hparam_search_encoder"
df = collect_hparam_records(base_dir)
summary = summarize_hparam_results(df)

print(summary)


    embed_dim  conv_hidden  val_mean   val_std  n_runs
6          16          128  0.007522  0.002141       3
9          32           64  0.008518  0.003507       3
7          16          256  0.009155  0.001284       3
5          16           64  0.009469  0.003935       3
11         32          256  0.009513  0.004044       3
10         32          128  0.009769  0.004113       3
1           8           64  0.011697  0.007318       3
4          16           32  0.012035  0.004845       3
8          32           32  0.012143  0.003920       3
0           8           32  0.014002  0.005891       3
2           8          128  0.014244  0.006438       3
3           8          256  0.021709  0.016053       3


diversity metrics

In [ ]:
from pathlib import Path
import sys

# Link to src/hparam_metrics.py
import pandas as pd
from src.temporal_autoencoder import TemporalWindowAutoEncoder
from src.hparam_metrics import share_metric_temporal

# ============================================================
# === 0. Configuration =======================================
# ============================================================
device = "cuda" if torch.cuda.is_available() else "cpu"

# === Paths to all models you want to compare ===
ckpt_paths = [
    "/content/gdrive/MyDrive/hparam_search_encoder/ed8_ch256_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed8_ch128_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed8_ch64_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed8_ch32_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed16_ch256_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed16_ch128_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed16_ch64_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed16_ch32_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed32_ch256_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed32_ch128_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed32_ch64_seed123_fold3.pt",
    "/content/gdrive/MyDrive/hparam_search_encoder/ed32_ch32_seed123_fold3.pt",
]

# === Mask for feature selection (kept as before) ===
mask_embed = torch.tensor(masks["mask_embed"], dtype=torch.bool, device=device)

# ============================================================
# === Helper: create sliding windows =========================
# ============================================================
def sliding_windows(x, window):
    """Generate sliding temporal windows [B, W, N, F]."""
    x = x[..., mask_embed]
    return torch.stack([x[t:t + window] for t in range(len(x) - window)], dim=0)

# ---------------------------------------------
# ===  Extract Fold-3 Validation Data  =========
# ---------------------------------------------
fold_index = 2   # fold3 because fold indices = [0,1,2]

_, val_data_f3 = extract_fold_data(
    temporal_node_tensor,
    folds,
    fold_index
)
val_data_f3 = torch.as_tensor(val_data_f3, dtype=torch.float32, device=device)

# ============================================================
# === 1. Evaluate each model =================================
# ============================================================

results = []

for ckpt_path in ckpt_paths:
    print(f"\n>>> Evaluating {ckpt_path}")

    # --- Load checkpoint ---
    ckpt = torch.load(ckpt_path, map_location=device)
    config = ckpt.get("config", {})

    # === Cloud embedding safety ===
    # If training config did not store mask_cloud, you must pass it manually
    config["mask_cloud"] = masks.get("mask_cloud", None)
    if config["mask_cloud"] is not None and not isinstance(config["mask_cloud"], torch.Tensor):
        config["mask_cloud"] = torch.tensor(config["mask_cloud"], dtype=torch.bool)

    # === Instantiate model with current API ===
    model = TemporalWindowAutoEncoder(
        in_dim=config["in_dim"],
        embed_dim=config["embed_dim"],
        window=config.get("window", 6),
        conv_hidden=config.get("conv_hidden", 64),
        dropout=config.get("dropout", 0.1),
        use_attention=config.get("use_attention", False),
        cloud_embed_dim=config.get("cloud_embed_dim", 3),
        mask_cloud=config.get("mask_cloud", None),
    ).to(device)

    # --- Load weights ---
    missing, unexpected = model.load_state_dict(ckpt["model_state"], strict=False)
    if missing or unexpected:
        print(f"ℹ️ State dict load: missing={len(missing)}, unexpected={len(unexpected)}")

    model.eval()
    window = config["window"]

    # --- Prepare validation windows ---
    val_seq = sliding_windows(val_data_f3.to(device), window=window)
    val_loader = torch.utils.data.DataLoader(val_seq, batch_size=8, shuffle=False)

    # --- Compute metrics ---
    metrics_all = []
    with torch.no_grad():
        for x_t in val_loader:
            x_t = x_t.to(device)
            _, z_t = model(x_t)
            metrics = share_metric_temporal(x_t, z_t)
            metrics_all.append(metrics)

    # === Aggregate overall metrics ===
    mean_input = torch.tensor([m["share_input"] for m in metrics_all]).mean().item()
    mean_embed = torch.tensor([m["share_embed"] for m in metrics_all]).mean().item()
    mean_delta = mean_embed - mean_input

    # === Aggregate mean component metrics ===
    mean_eff_rank_in  = torch.tensor([m["mean_details"]["eff_rank_in"]  for m in metrics_all]).mean().item()
    mean_cos_div_in   = torch.tensor([m["mean_details"]["cos_div_in"]   for m in metrics_all]).mean().item()
    mean_rel_var_in   = torch.tensor([m["mean_details"]["rel_var_in"]   for m in metrics_all]).mean().item()

    mean_eff_rank_out = torch.tensor([m["mean_details"]["eff_rank_out"] for m in metrics_all]).mean().item()
    mean_cos_div_out  = torch.tensor([m["mean_details"]["cos_div_out"]  for m in metrics_all]).mean().item()
    mean_rel_var_out  = torch.tensor([m["mean_details"]["rel_var_out"]  for m in metrics_all]).mean().item()

    results.append({
        "model": ckpt_path.split("/")[-1].replace(".pt", ""),
        "share_input": mean_input,
        "share_embed": mean_embed,
        "delta": mean_delta,
        "r_in": mean_eff_rank_in,
        "c_in": mean_cos_div_in,
        "v_in": mean_rel_var_in,
        "r_out": mean_eff_rank_out,
        "c_out": mean_cos_div_out,
        "v_out": mean_rel_var_out,
        "attention": config.get("use_attention", False)
    })

# ============================================================
# === 2. Display Comparison ==================================
# ============================================================
df = pd.DataFrame(results)
df = df.round(4)
df = df.sort_values("delta", ascending=False)

print("\n=== Representation Diversity Comparison ===")
print(df[[
    "model", "attention", "share_input", "share_embed", "delta",
    "r_in", "c_in", "v_in", "r_out", "c_out", "v_out"
]].to_string(index=False))
